## 天候による売り上げ予測(回帰) 

回帰モデル　
グループを予測するのではなく、物の販売予測や利用者人数を予測するためのモデル


### 共通事前処理

In [ ]:
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

In [2]:
# 共通事前処理

# 余分なワーニングを非表示にする
import warnings
warnings.filterwarnings('ignore')

# 必要ライブラリのimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# matplotlib日本語化対応
import japanize_matplotlib

# データフレーム表示用関数
from IPython.display import display

# 表示オプション調整
# numpyの浮動小数点の表示精度
np.set_printoptions(suppress=True, precision=4)

# pandasでの浮動小数点の表示精度
pd.options.display.float_format = '{:.4f}'.format

# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)

# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14

# 乱数の種
random_seed = 123

オリジナルURL   
https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

#### 1.1 データ読み込み


In [ ]:
# ダウンロード元URL
url = 'https://archive.ics.uci.edu/ml/\
machine-learning-databases/00275/\
Bike-Sharing-Dataset.zip'

# 公開データのダウンロードと解凍
!wget $url -O Bike-Sharing-Dataset.zip | tail -n 1
!unzip -o Bike-Sharing-Dataset.zip | tail -n 1

#### データ項目メモ

instant インデックス  
dteday 日付(yy-mm-dd)  
season 季節 (1: 冬 2: 春 3: 夏 4:秋)  
yr 年 (0: 2011, 1:2012)  
mnth 月  (1 - 12)  
hr 時間  (0 - 23)  
holiday 祝日  
weekday 曜日 (0 - 6)  
workingday  勤務日 (1: 勤務日 0: 休日)  
weathersit 天気 (1: 晴れから曇り 2: 霧 3: 小雨 4: 大雨)  
temp 気温 (正規化済み)  
atemp 体感気温 (正規化済み)  
hum 湿度 (正規化済み)  
windspeed 風速 (正規化済み)  
casual 臨時利用者数  
registered 登録利用者数  
cnt 全体利用者数  

In [ ]:
# データの状態確認
!head -5 day.csv

In [ ]:
# day.csvをデータフレームに取り込み
# 日付を表す列はparse_datesで指定する
df = pd.read_csv('day.csv', parse_dates=[1])

# データ属性の確認
print(df.dtypes)

In [6]:
# instant は連番で予測で不要なので削除
df = df.drop('instant', axis=1)

# 項目名の日本語化
columns = [
    '日付',  '季節',  '年', '月', '祝日', '曜日', '勤務日', '天気', 
    '気温', '体感温度',  '湿度', '風速',
    '臨時ユーザー利用数', '登録ユーザー利用数', '全体ユーザー利用数'
]

# 項目名を日本語に置き換え
df.columns = columns

####1.2 データ確認

In [ ]:
# 先頭5行の確認
display(df.head())

In [ ]:
# 欠損値チェック
df.isnull().sum()

In [ ]:
# 度数分布表示

# グラフのサイズ調整のためのおまじない
from pylab import rcParams
rcParams['figure.figsize'] = (12, 12)

# データフレームの数値項目でヒストグラム表示
df.hist(bins=20)
plt.tight_layout()
plt.show()

In [ ]:
#統計情報の調査
df.describe()

#### 時系列データのグラフ表示

In [ ]:
# 時系列グラフの描画 (登録利用者数)
plt.figure(figsize=(12,4))

# グラフ描画
plt.plot(df['日付'],df['登録ユーザー利用数'],c='b')

# 方眼表示など
plt.grid()
plt.title('登録ユーザー利用数')

# 画面出力
plt.show()

####2.1 データ前処理
One-Hot表現に変更する

get_dummiesを使ってOne-Hot表現に変更する関数を作成して利用する。\
変更する項目は['祝日']

In [12]:
# One-Hot表現に変換する関数
def enc(df, column):
  df_dummy = pd.get_dummies(df[column],prefix=column)

  df_drop = df.drop([column], axis=1)

  df1 = pd.concat([df_drop,df_dummy], axis=1)
  
  return df1

In [ ]:
#祝日をOne-Hot表現に変更する
df1 = enc(df, '祝日')

# 先頭5行の確認
display(df1.head())

In [ ]:
#月をOne-Hot表現に変更する
df2 = enc(df1, '月')

# 先頭5行の確認
display(df2.head())

In [ ]:
#曜日をOne-Hot表現に変更する
df3 = enc(df2, '曜日')

# 先頭5行の確認
display(df3.head())

In [ ]:
#天気をOne-Hot表現に変更する
df4 = enc(df3, '天気')

# 先頭5行の確認
display(df4.head())

In [ ]:
#季節をOne-Hot表現に変更する
df5 = enc(df4, '季節')

# 先頭5行の確認
display(df5.head())

#### 2.2 データ分割

回帰モデルは過去のデータから未来を予測するので、日付を境に訓練データと検証データを分ける

方針
* 目的変数yは「登録ユーザー利用数」とする
* 「全体ユーザー利用数」と「臨時ユーザー利用数」は対象から落とす
* 日付も入力変数として不要なので入力変数xから落とす
* 時間軸に関しては 2012-11-01より前を訓練データ、後ろを検証データとする
  2年分のデータがあるので、訓練データ22ヵ月、検証データ2か月とする

  0.9


In [18]:
# x, yへの分割
x = df5.drop(['日付', '臨時ユーザー利用数', '登録ユーザー利用数',
    '全体ユーザー利用数'], axis=1)
y = df5['登録ユーザー利用数'].values

In [19]:
# 分割日 mdayの設定
mday = pd.to_datetime('2012-11-1')

# 訓練用indexと検証用indexを作る
train_index = df5['日付'] < mday
test_index = df5['日付'] >= mday

# 入力データの分割
x_train = x[train_index]
x_test = x[test_index]

# yも同様に分割
y_train = y[train_index]
y_test = y[test_index]

# 日付データの分割(グラフ表示用)
dates_test = df5['日付'][test_index]

In [ ]:
# 結果確認(サイズを確認)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
print(dates_test)

# 結果確認 (境界値を重点的に)
display(x_train.tail())
display(x_test.head())

In [ ]:
# 目的変数の分割結果確認
print(y_train[:10])

### 4 チューニング

#### 4.1 アルゴリズムのハイパーパラメータ設定

In [ ]:
pip install scikit-optimize

In [64]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from skopt import BayesSearchCV

from sklearn.model_selection import KFold
kfold = KFold(n_splits=3)

from sklearn.model_selection import StratifiedKFold
stratifiedkfold = StratifiedKFold(n_splits=3)

In [66]:
def gridsearchcv(algorithm, param, cv, x_train, y_train, x_test, y_test ):

  # アルゴリズム名を取得する
  name = algorithm.__class__.__name__
  print(f'{name} GridSearchCV')

  # グリットサーチを定義する
  searchcv = GridSearchCV(algorithm, param, cv=cv)
  
  # 学習
  searchcv.fit(x_train, y_train)

  # 交差検証法でベスト  
  best = searchcv.best_estimator_

  # 予測
  best_pred = best.predict(x_test)
  print(best)

  # 評価(登録ユーザー利用数)
  score = best.score(x_test, y_test)
  print(f'{name} score: {score:.4f}')
  
  return best_pred, score

In [67]:
def randamsearchcv(algorithm, param, cv, x_train, y_train, x_test, y_test ):
  # アルゴリズム名を取得する
  name = algorithm.__class__.__name__
  print(f'{name} RandomizedSearchCV')

  # グリットサーチを定義する
  searchcv = RandomizedSearchCV(algorithm, param, cv=cv)
  
  # 学習
  searchcv.fit(x_train, y_train)

  # 交差検証法でベスト  
  best = searchcv.best_estimator_

  # 予測
  best_pred = best.predict(x_test)
  print(best)

  # 評価(登録ユーザー利用数)
  score = best.score(x_test, y_test)
  print(f'{name}  score: {score:.4f}')
  
  return best_pred, score

In [68]:
def bayessearchcv(algorithm, param, cv, x_train, y_train, x_test, y_test ):

    # アルゴリズム名を取得する
    name = algorithm.__class__.__name__
    print(f'{name} BayesSearchCV')
  
    # ベイズ最適化を定義する
    searchcv = BayesSearchCV(estimator=algorithm, search_spaces=param, cv=cv, n_iter = 30)

    # 学習
    searchcv.fit(x_train, y_train)

    # 交差検証法でベスト  
    best = searchcv.best_estimator_

    # 予測
    best_pred = best.predict(x_test)
    print(best)

    # 評価(登録ユーザー利用数)
    score = best.score(x_test, y_test)
    print(f'{name} score: {score:.4f}')

    return best_pred, score

In [69]:
def plotresult(dates_test,best_pred, score, y_test):
  #正解データと予測結果を散布図で比較 (登録ユーザー利用数)
  plt.figure(figsize=(6,6))
  y_max = y_test.max()
  plt.plot((0,y_max), (0, y_max), c='k')
  plt.scatter(y_test, best_pred, c='b')
  plt.title(f'正解データと予測結果の散布図(登録ユーザー利用数)\
    R2={score:.4f}')
  plt.grid()
  plt.show()

  # 時系列グラフの描画 (登録ユーザー利用数)
  import matplotlib.dates as mdates
  fig, ax = plt.subplots(figsize=(8, 4))

  # グラフ描画
  ax.plot(dates_test, y_test, label='正解データ', c='k')
  ax.plot(dates_test, best_pred, label='予測結果', c='b')

  # 日付目盛間隔
  # 木曜日ごとに日付を表示
  weeks = mdates.WeekdayLocator(byweekday=mdates.TH)
  ax.xaxis.set_major_locator(weeks)

  # 日付表記を90度回転
  ax.tick_params(axis='x', rotation=90)

  # 方眼表示など
  ax.grid()
  ax.legend()
  ax.set_title('登録ユーザー利用数予測')

  # 画面出力
  plt.show()  


In [76]:
# アルゴリズム選定

# 線形重回帰
from sklearn import linear_model
algorithm1 = linear_model.LinearRegression()

param1 = {}

# サポートベクターカーネル回帰
from sklearn.svm import SVR
algorithm2 = SVR(kernel='rbf')

param2={
    'C' : [10 ** i for i in range(-5, 6)],
    'gamma' : [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 2.0]
}

# 回帰木
from sklearn.tree import DecisionTreeRegressor
algorithm3 = DecisionTreeRegressor(max_leaf_nodes = 20)

param3 =  {'criterion': ['mse', 'mae'],
            'splitter': ['best', 'random'],
            'max_depth': [i for i in range(1, 11)],
            'min_samples_split': [i for i in range(2, 11)],
            'min_samples_leaf': [i for i in range(1, 11)]
           }

# ランダムフォレスト回帰
from sklearn.ensemble import RandomForestRegressor
# モデル構築、パラメータはデフォルト
algorithm4 = RandomForestRegressor()

param4 = {'n_estimators': [i for i in range(1, 21)],
          'criterion': ['mse', 'mae'],
          'max_depth': [i for i in range(1, 11)]
          }


# XGBoot回帰
from xgboost import XGBRegressor
algorithm5 = XGBRegressor(objective ='reg:squarederror', random_state=0)

param5 = {'learning_rate':[0.1, 0.3, 0.5, 0.03, 0.003],
          'gamma' : [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 2.0],
          'max_depth': [i for i in range(1, 11)],
          'subsample':[0.5, 0.8, 0.9, 1],
          'colsample_bytree': [0.5,1.0]
         }


# LightGBM回帰
from lightgbm import LGBMRegressor
algorithm6 = LGBMRegressor(random_state=0)

param6 = {'num_leaves':[7, 14, 21, 28, 31, 50],
          'learning_rate':[0.1, 0.3, 0.5, 0.03, 0.003],
          'n_estimators':[50, 100, 200, 500],    
          'max_depth':[i for i in range(1, 11)]
         }

algorithms = [algorithm1,algorithm2,algorithm3,algorithm4,algorithm5,algorithm6]

params = [ param1, param2, param3, param4, param5, param6 ]


### 4.2 グリットサーチ, ランダムサーチ, ベイズ最適化　 学習・予測・評価

In [ ]:
i = 0

# 登録ユーザー利用数予測モデルの学習と予測
for algorithm in algorithms:

  print(f'============== start { algorithm.__class__.__name__} =================')

  # サーチするパラメータ表示
  param = params[i]
  print(param)

  ## グリットサーチ
  #best_pred, score = gridsearchcv(algorithm, param, stratifiedkfold, x_train, y_train, x_test, y_test)
  #plotresult(dates_test,best_pred, score, y_test)

  if i > 0:
    # ランダムサーチ
    best_pred, score = randamsearchcv(algorithm, param, stratifiedkfold, x_train, y_train, x_test, y_test)
    plotresult(dates_test,best_pred, score, y_test)

    ## ベイズ最適化サーチ
    best_pred, score = bayessearchcv(algorithm, param, stratifiedkfold, x_train, y_train, x_test, y_test)
    plotresult(dates_test,best_pred, score, y_test)

  print(f'============== end   { algorithm.__class__.__name__} =================')

  i = i + 1


SVR RandomizedSearchCV

SVR(C=100000, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
SVR  score: 0.6883 r2_ score: 0.6883


{'learning_rate': [0.1, 0.3, 0.5], 'gamma': [1e-08, 1e-07, 1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1, 1.0, 2.0], 'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'subsample': [0.5, 0.8, 0.9, 1], 'colsample_bytree': [0.5, 1.0]}

XGBRegressor RandomizedSearchCV

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0.0001,
             importance_type='gain', learning_rate=0.3, max_delta_step=0,
             max_depth=2, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=123, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)
XGBRegressor  score: 0.7112 r2_ score: 0.7112